In [ ]:
import sys

sys.path.append("../..")

import torch
from fourier_scaffold import (
    HadamardShiftMatrix,
    HadamardShift,
    HadamardShiftRat,
    HadamardShiftMatrixRat,
    FourierScaffold,
    GuassianFourierSmoothingMatrix
)

shapes = torch.tensor([(3, 3), (5, 5)])
shift_rat = HadamardShiftMatrixRat(shapes=shapes)
shift_nonrat = HadamardShiftMatrix()

In [ ]:
scaffold = FourierScaffold(
    shapes=shapes,
    D=400,
    shift=shift_rat,
    _skip_K_calc=True,
    _skip_gs_calc=True,
    representation='matrix'
)

In [ ]:
scaffold.P = scaffold.zero()

In [ ]:
scaffold.velocity_shift(torch.tensor([-2,0]))

In [ ]:
scaffold.sharpen()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(scaffold.get_all_probabilities())
plt.colorbar()

In [ ]:
def combine_additive(P1, P2, alpha=0.5):
    P = (1 - alpha) * (P1) + alpha * P2
    return P


def combine_multiplicative(P1, P2):
    S = (P1 * P2.conj()).sum().abs()
    P = P1 @ P2.H
    return P / S

In [ ]:
scaffold = FourierScaffold(
    shapes=shapes,
    D=400,
    shift=shift_rat,
    smoothing=GuassianFourierSmoothingMatrix([10]*2, [1]*2),
    _skip_gs_calc=True,
    representation='matrix'
)

P_original = scaffold.P.clone()
scaffold.velocity_shift(torch.tensor([0.5,0]))
P_smoothed = scaffold.smoothing(scaffold.P)
P_combined = combine_multiplicative(P_original, P_smoothed)

In [ ]:
import matplotlib.pyplot as plt
from graph_utils import plot_imgs_side_by_side

fig, ax = plt.subplots(1, 3, figsize=(15,9))
plot_imgs_side_by_side(
  imgs=[scaffold.get_all_probabilities(P) for P in [P_original, P_smoothed, P_combined]],
  titles=['original', 'shifted', 'combined'],
  axs=ax,
  fig=fig,
  use_first_img_scale=False
)